# ACTINN-Pytorch Tutorial: Classifying NeuroCOVID scRNAseq

In this notebook, we will go over how to load a pre-trained ACTINN model, and how to make predictions and evaluate the performance

## Load in pre-trained ACTINN model

Since our implementation is in pytorch, we can use the `load` funtion that pytorch provides. Our model is stored as a dict, with `epoch` corresponding to the current epoch, and `Saved_Model` corresponding to the model.

In [1]:
import torch 

model_dict = torch.load("/home/ubuntu/SindiLab/ACTINN-PyTorch/ClassifierWeights/NeruoCOVID_model_epoch_50_iter_0.pth")

actinn = model_dict["Saved_Model"]

print(actinn)

Classifier(
  (classifier_sequential): Sequential(
    (0): Linear(in_features=22807, out_features=100, bias=True)
    (1): ReLU()
    (2): Linear(in_features=100, out_features=50, bias=True)
    (3): ReLU()
    (4): Linear(in_features=50, out_features=25, bias=True)
    (5): ReLU()
    (6): Linear(in_features=25, out_features=20, bias=True)
  )
)


### Determine the device where you want to generate data from

We recommend using GPUs for *training*, but for inference either CPUs or GPUs should work just fine (but GPUs would be faster). 

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if str(device) == "cuda":
    print('Using GPU (CUDA)')
else:
    print('Using CPU')

Using GPU (CUDA)


## Load in Data

Let us load in the test data set now

In [3]:
from ACTINN.utils import *
from ACTINN import Scanpy_IO

In [4]:
_, test_data_loader = Scanpy_IO('/home/ubuntu/RawData/78KNeuroCOVID_preprocessed_splitted_logged.h5ad',
                                                        test_no_valid = True,
                                                        log=False,
                                                        verbose = 1)

==> Reading in Scanpy/Seurat AnnData
    -> Splitting Train and Validation Data
==> Using cluster info for generating train and validation labels
==> Checking if we have sparse matrix into dense


/home/ubuntu/anaconda3/lib/python3.7/site-packages/pandas/core/arrays/categorical.py:2487: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)


==> sample of the training data: tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
==> sample of the test data: tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])


## Use the `evaluate_classifier` function for a full report!

We provide easy-to-use utilities for making things easier. One of these utility functions is `evaluate_classifier`, which provides a full classification report if wanted (using `sklearn`). (valid_data_loader, model,classification_report=False)


In [6]:
evaluate_classifier(test_data_loader, actinn, classification_report=True)

==> Evaluating on Validation Set:
    -> Accuracy of classifier network on validation set: 97.9054 %
    -> Non-Weighted F1 Score on validation set: 0.9697 
    -> Weighted F1 Score on validation set: 0.9790 
              precision    recall  f1-score   support

         0.0       0.99      0.99      0.99      2067
         1.0       0.99      1.00      0.99      1077
         2.0       0.96      0.96      0.96       781
         3.0       0.96      0.96      0.96       780
         4.0       0.97      0.97      0.97       760
         5.0       0.99      0.98      0.98       514
         6.0       0.98      0.98      0.98       366
         7.0       0.99      0.99      0.99       345
         8.0       0.98      0.99      0.99       297
         9.0       1.00      0.99      1.00       295
        10.0       0.99      0.97      0.98       159
        11.0       0.98      0.93      0.96       137
        12.0       0.98      0.90      0.94       102
        13.0       0.92      0.94 